In [0]:
!git clone https://github.com/jjmachan/DeepHash.git
import DeepHash.datasets

# Task 3
## Training Using the Triplet Loss


In [0]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
from torch import optim
import numpy as np
from torch.optim import lr_scheduler
import torch.optim as optim

from DeepHash.trainer import fit
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from DeepHash.utils import freeze_model, list_trainable, del_last_layers, save, load, create_embeddings

In [0]:
batch_size = 64
num_workers = 1

#define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# load imagenet
image_dataset = {
        'train' :datasets.CIFAR10('./', train=True, download=True, transform=data_transforms['train']),
        'test' : datasets.CIFAR10('./', train=False, download=True, transform=data_transforms['train']) 
}

# Create the dataloaders
data_loader = {
    'train': torch.utils.data.DataLoader(image_dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'test': torch.utils.data.DataLoader(image_dataset['test'], batch_size=batch_size, shuffle=False, num_workers=num_workers)
}

In [0]:
# An identity layer to pass the fc layer in resnet
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
  
# Define model
resnet18  = models.resnet18(pretrained=True)
resnet18.fc = Identity()

# Freeze all the parameters in the model
def freeze_model(model):
  for params in model.parameters():
    params.requires_grad=False

# freeze
#freeze_model(vgg19)

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(triplet_train_loader)
images, labels = dataiter.next()

print(images[0].shape)
# show images
imshow(images[0][0])
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [0]:
# Set up data loaders
from DeepHash.datasets import TripletCifar1
from triplet.datasets import TripletMNIST

cuda = torch.cuda.is_available()
triplet_train_dataset = TripletCifar1(image_dataset['train']) # Returns triplets of images
triplet_test_dataset = TripletCifar1(image_dataset['test'])
batch_size = 32
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from triplet.networks import EmbeddingNet, TripletNet
from triplet.losses import TripletLoss

margin = 2.
embedding_net = resnet18
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 10
log_interval = 500

In [0]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

In [0]:
torch.save(model, './triplet_resnet18_2.mdl')

In [0]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

In [0]:
torch.save(model, './triplet_vgg20_3.mdl')

In [0]:
# save to google drive 

from google.colab import drive
drive.mount('/gdrive')

!cp *.mdl /gdrive/My\ Drive/tooploox

In [0]:
# load data
from google.colab import drive
drive.mount('/gdrive')
!cp /gdrive/My\ Drive/tooploox/*.mdl ./ 

In [0]:
model_test = torch.load('./triplet_resnet18_2.mdl')
model.eval()

In [0]:
# Freeze all the parameters in the model
def freeze_model(model):
  for params in model.parameters():
    params.requires_grad=False

# check if all the parameters have been freezed
def list_trainable(model):
  for params in model.parameters():
    print(params.requires_grad)
  
# delete the last layers
def del_last_layers(model_class, num_layers):
  model_class = nn.Sequential(*list(model_class.children())[:-num_layers])
  return model_class

In [0]:
def create_embeddings(model, embedding_size):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print(device)

  features = {}
  targets = {}
  model.to(device)
  features['train'] = np.empty([0, embedding_size])
  targets['train'] = np.empty([0, ])

  features['test'] = np.empty([0, embedding_size])
  targets['test'] = np.empty([0,])

  for i, (images,target) in enumerate(data_loader['train']):
    images = images.to(device)
    target = target.to(device)

    try:
      output = model(images).cpu().numpy()
      features['train'] = np.append(features['train'],output, axis=0)
      targets['train'] = np.append(targets['train'],target.cpu(), axis=0)
    except:
      print(output.shape)
      print('error occured: ')
      return (None, None)
      
    if i%100 == 0:
      print(i)

  for i, (images,target) in enumerate(data_loader['test']):
    images = images.to(device)
    target = target.to(device)

    output = model(images).cpu().numpy()
    features['test'] = np.append(features['test'],output, axis=0)
    targets['test'] = np.append(targets['test'],target.cpu(), axis=0)

    if i%100 == 0:
      print(i)
  return (features, targets)

In [0]:
model_emb = model.embedding_net
freeze_model(model_emb)
print(model_emb)

In [0]:
from DeepHash.utils import create_embeddings
features, targets = create_embeddings(model_emb, data_loader, 512)


In [0]:
# save the computed embeddings
save(features, targets, 'resnet18_triplet_margin2')

In [0]:
# save to google drive 

from google.colab import drive
drive.mount('/gdrive')

!cp *.embs /gdrive/My\ Drive/tooploox

In [0]:
!cp /gdrive/My\ Drive/tooploox/*.embs .

In [0]:
!ls /gdrive/My\ Drive/tooploox

In [0]:
features, target = load('resnet18_triplet')

# Classification Accuracy 

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k_range = range(25,30)

def search_knn_accuracies(k_range, features, targets):
  acc = []
  for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(features['train'], targets['train'])
    #print('finished fitting')
    predict = knn.predict(features['test'][:300,:])
    #print('predicted')
    score = metrics.accuracy_score(targets['test'][:300], predict)
    print('K value: %d, accuracy: %0.7f' %(k, score))
    acc.append(score)
  return acc


# the best score was obtained when k = 20:24
acc = search_knn_accuracies(k_range, features, target)
print('final Accuracy: ',sum(acc)/len(acc))

# t-SNE plots

In [0]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from sklearn.manifold import TSNE
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
RS = 123
# need to create a subset of the data, too much time to process otherwise
x_subset = features['train'][:2000]
y_subset = targets['train'][:2000]

print(np.unique(y_subset))
labels = {
     0: 'airplane',  
     1: 'automobile',
     2: 'bird',
     3: 'cat',
     4: 'deer',
     5: 'dog',
     6: 'frog',
     7: 'horse',
     8: 'ship',
     9: 'truck',
}
# Utility function to visualize the outputs of PCA and t-SNE

def fashion_scatter(x, colors):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(labels[i]), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [0]:
# do pca before passing to tsne to reduce noice and fast performance
time_start = time.time()

pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(x_subset)

print('PCA with 50 components done! Time elapsed: {} seconds'.format(time.time()-time_start))

print('Cumulative variance explained by 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))
# perform tsne on 50 components
time_start = time.time()


fashion_pca_tsne = TSNE(random_state=RS).fit_transform(pca_result_50)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [0]:
fashion_scatter(fashion_pca_tsne, y_subset)

In [0]:
one = features['train'][0].reshape(1,-1)
one_label = target['train'][0]
print(one_label)

In [0]:
a = np.linalg.norm(features['test']-one, axis=1)
a.shape

In [0]:
new_a = np.append(a.reshape(-1,1), target['test'].reshape(-1,1), axis=1)
new_a = new_a.tolist()
new_a[:20]

In [0]:
def take_fist(elem):
  return elem[0]

sorted_a = sorted(new_a, key=take_fist)
sorted_a = np.array(sorted_a)


In [0]:
one_label_map = target['test'] == one_label
sorted_a = one_label_map*sorted_a[:,0]
sorted_a[:90]

In [0]:
for i, num in enumerate(sorted_a):
  print(i, num)